In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
# Load API Credentials
with open('/Users/sharheatherclark/.secret/yelp_api.json') as f:   #use your path here!
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api


In [3]:
# set our API call parameters and define our search
LOCATION = 'NY,NY'
TERM = 'Pizza'

In [4]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "Data/results_in_progress_NY_pizza.json"
JSON_FILE


'Data/results_in_progress_NY_pizza.json'

In [5]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")


[i] Data/results_in_progress_NY_pizza.json not found. Saving empty list to file.


In [6]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    # this code above renames file previous results instead of JSON_FILE
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')



- 0 previous results found.


In [7]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()



dict_keys(['businesses', 'total', 'region'])

In [8]:
## How many results total?
total_results = results['total']
total_results



16200

In [9]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page



20

There are over 11000 businesses to retrieve from our API and we can get 20 results at a time (per "page").
- We can calculate the # of results remaining by subtracting our offset (length of our previous results) from our total.
- Then we can determining how many pages we will need by dividing the results by 20 (or whatever the value happens to be for results per page)
- Note that we need to round up the number of pages in order to get all of the results. Even if there is only 1 result on the last page, we want to include that page! To do this we will use math.ceil.

In [10]:
# Import additional packages for controlling our loop
import time, math

# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

#This code/output says that we need 820 pages to get all the data we want

810

You can see that having to manually go through 560 pages would be quite time consuming and inefficient! First we are going to save the first page into our file, and then we will add on to it with a for loop.

In [11]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)


In [12]:
#showing the tqdm working
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 
#code above is looping through a pause every 820 bc range is n_pages

  0%|          | 0/810 [00:00<?, ?it/s]

In [13]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    # add a 200ms pause
    time.sleep(.2)



  0%|          | 0/810 [00:00<?, ?it/s]

YelpAPIError: VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.

In [14]:
## delete file and confirm it no longer exits.
os.remove(JSON_FILE)
os.path.isfile(JSON_FILE)

False

In [34]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)  

In [35]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages



[i] Data/results_in_progress_NY_pizza.json not found. Saving empty list to new file.
- 0 previous results found.


815

In [36]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)



  0%|          | 0/815 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [37]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,zj8Lq1T8KIC5zwFief15jg,prince-street-pizza-new-york-2,Prince Street Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/PfI8oV...,False,https://www.yelp.com/biz/prince-street-pizza-n...,4864,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 40.72308755605564, 'longitude': -...","[delivery, pickup]",$,"{'address1': '27 Prince St', 'address2': None,...",+12129664100,(212) 966-4100,2209.311618
1,ysqgdbSrezXgVwER2kQWKA,julianas-brooklyn-3,Juliana's,https://s3-media2.fl.yelpcdn.com/bphoto/NVoLFl...,False,https://www.yelp.com/biz/julianas-brooklyn-3?a...,2634,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 40.70274718768062, 'longitude': -...",[delivery],$$,"{'address1': '19 Old Fulton St', 'address2': '...",+17185966700,(718) 596-6700,1289.857286
2,WG639VkTjmK5dzydd1BBJA,rubirosa-new-york-2,Rubirosa,https://s3-media3.fl.yelpcdn.com/bphoto/F65qqO...,False,https://www.yelp.com/biz/rubirosa-new-york-2?a...,2985,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 40.722766, 'longitude': -73.996233}",[pickup],$$,"{'address1': '235 Mulberry St', 'address2': ''...",+12129650500,(212) 965-0500,2268.491950
3,v1DHGRNCH9247WLYoaoA9A,l-industrie-pizzeria-brooklyn,L'industrie Pizzeria,https://s3-media3.fl.yelpcdn.com/bphoto/KKB2Tr...,False,https://www.yelp.com/biz/l-industrie-pizzeria-...,934,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 40.71162, 'longitude': -73.95783}",[delivery],$,"{'address1': '254 S 2nd St', 'address2': '', '...",+17185990002,(718) 599-0002,1868.295512
4,KEEXuHCYhsQJ-zpJhu98bA,joes-pizza-new-york-148,Joe's Pizza,https://s3-media1.fl.yelpcdn.com/bphoto/VkqlwR...,False,https://www.yelp.com/biz/joes-pizza-new-york-1...,1870,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 40.7547, 'longitude': -73.98696}","[delivery, pickup]",$,"{'address1': '1435 Broadway', 'address2': '', ...",+16465594878,(646) 559-4878,5345.192429


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,PgsOT5QnYgXYUVSR_-prYA,kiss-my-slice-new-york-5,Kiss My Slice Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/Uk89PY...,False,https://www.yelp.com/biz/kiss-my-slice-new-yor...,50,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 40.75842, 'longitude': -73.96261}","[pickup, delivery]",NaN,"{'address1': '401 E 57th St', 'address2': '', ...",+12129357000,(212) 935-7000,5888.038310
996,SdJpihBeGYcUVe5bflTMWg,acqua-e-farina-pizzeria-syosset-4,Acqua e Farina Pizzeria,https://s3-media2.fl.yelpcdn.com/bphoto/JvEif-...,False,https://www.yelp.com/biz/acqua-e-farina-pizzer...,59,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 40.827750587448, 'longitude': -73...",[delivery],NaN,"{'address1': '43 Berry Hill Rd', 'address2': '...",+15168022272,(516) 802-2272,42320.603487
997,zvQLNPA8vh1Yf4mhZtjuSQ,genovas-pizza-and-pasta-long-beach,Genovas Pizza & Pasta,https://s3-media4.fl.yelpcdn.com/bphoto/wedtv2...,False,https://www.yelp.com/biz/genovas-pizza-and-pas...,17,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 40.58653, 'longitude': -73.68675}","[pickup, delivery]",NaN,"{'address1': '212 New York Ave', 'address2': N...",+15166822824,(516) 682-2824,28091.686690
998,OyfPqhWrNhxFb_XGpcnUHw,pizza-time-brooklyn,Pizza Time,https://s3-media3.fl.yelpcdn.com/bphoto/ahNGZo...,False,https://www.yelp.com/biz/pizza-time-brooklyn?a...,71,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 40.62494, 'longitude': -73.96244}",[],$,"{'address1': '1324 Ave J', 'address2': '', 'ad...",+17182528801,(718) 252-8801,9231.463796
999,qoc8jyoCBW4tw9zspA2K6Q,jeannies-new-york-3,Jeannie's,https://s3-media3.fl.yelpcdn.com/bphoto/IbhxaD...,False,https://www.yelp.com/biz/jeannies-new-york-3?a...,24,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 40.7662427, 'longitude': -73.9812...",[],$$,"{'address1': '225 W 57th St', 'address2': None...",+12122952185,(212) 295-2185,6592.236289


In [38]:
# check for duplicate results
final_df.duplicated().sum()

TypeError: unhashable type: 'list'

In [39]:
# check for duplicate ID's 
final_df.duplicated(subset='id').sum()

211

In [40]:
## Drop duplicate ids and confirm there are no more duplicates
final_df = final_df.drop_duplicates(subset='id')
final_df.duplicated(subset='id').sum()

0

In [41]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_NY_pizza.csv.gz', compression='gzip',index=False)